In [1]:
#Netflows are total count of occurence of a src_ip in a particular window
import pandas as pd
import numpy as np
#Edit path to the file for extracting features
def getFeatures(file_path):
    df=pd.read_csv(file_path)
    temp = pd.get_dummies(df.Tag)
    df['attack'] = temp.Attack
    df['normal'] = temp.Normal
    del temp, df['Tag']
    group = df.groupby(['bucket','source'])
    features = group.sourcePort.nunique()
    features = pd.DataFrame(features)
    features.rename(columns={'sourcePort':'usrc_port'},inplace=True)
    features['udest_ip'] = group.destination.nunique()
    features['udest_port'] = group.destinationPort.nunique()
    features['netflows'] = group.destination.count()
    features['bytes'] = group.totalSourceBytes.sum()
    features['packets'] = group.totalSourcePackets.sum()
    features['attack'] = group.attack.sum()
    features['normal'] = group.normal.sum()
    features.reset_index(inplace=True) 
    return features
    
def featurestoArray(features):
    #Converting DF to np array
    features_array = features.copy()
    del features_array['attack'],features_array['normal'],features_array['source'],features_array['bucket']
    features_array = features_array.as_matrix()
    return features_array

def getClusterFeatures(labeled_features):
    group = labeled_features.groupby(['label'])
    clusterfeatures = group.bucket.count()
    clusterfeatures = pd.DataFrame(clusterfeatures)
    clusterfeatures.rename(columns={'bucket':'instances'},inplace=True) 
    clusterfeatures['netflows']=group.netflows.sum()
    clusterfeatures['avgnetflows']=group.netflows.mean()
    clusterfeatures['stdnetflows']=group.netflows.std()
    clusterfeatures['usrc_ip']=group.source.nunique()
    clusterfeatures['avgsrc_port']=group.usrc_port.mean()
    clusterfeatures['stdsrc_port']=group.usrc_port.std()
    clusterfeatures['avgdest_ip']=group.udest_ip.mean()
    clusterfeatures['stddest_ip']=group.udest_ip.std()
    clusterfeatures['avgdest_port']=group.udest_port.mean()
    clusterfeatures['stddest_port']=group.udest_port.std()
    clusterfeatures['avgbytes']=group.bytes.mean()
    clusterfeatures['stdbytes']=group.bytes.std()
    clusterfeatures['avgpackets']=group.packets.mean()
    clusterfeatures['stdpackets']=group.packets.std()
    clusterfeatures['attack']=group.attack.sum()
    clusterfeatures['normal']=group.normal.sum()
    #True = Botnet    False = Normal
    clusterfeatures['label'] = (group.attack.sum()/group.normal.sum()>0.01)
    return clusterfeatures

In [2]:
def getLabeledFeaturesGMM(features_array):
    X_train = features_array
    from sklearn import mixture
    #Number of clusters = number of components
    # Four covariance Type ['spherical', 'diag', 'tied', 'full'] iterations can be changed
    clf = mixture.GMM(n_components=40, covariance_type='full', n_iter=100)
    label = clf.fit_predict(X_train)
    return label           

def addLabelTofeatures(features, label):
    labeled_features = features.copy()
    labeled_features['label'] = label
    return labeled_features

In [3]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Imputer 
features = getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/NDF/ndf3.csv")
X_train, X_test = train_test_split(features, test_size=0.3,random_state=0)
#X_train = getFeatures(file_path="F:/MTP/working_directory/ndf3.csv")
#X_train

/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
feature_array_train = featurestoArray(features=X_train)
train_label = getLabeledFeaturesGMM(features_array=feature_array_train)
X_train = addLabelTofeatures(features=X_train,label=train_label)
train_cluster_features = getClusterFeatures(labeled_features=X_train) 
train_labels = train_cluster_features['label']
del train_cluster_features['attack'],train_cluster_features['normal'],train_cluster_features['label']
train_cluster_features = Imputer.fit_transform(Imputer(strategy="most_frequent",axis=0),train_cluster_features)

feature_array_test = featurestoArray(features=X_test)
test_label = getLabeledFeaturesGMM(features_array=feature_array_test)
X_test = addLabelTofeatures(features=X_test,label=test_label)
test_cluster_features = getClusterFeatures(labeled_features=X_test)
test_labels = test_cluster_features['label']
del test_cluster_features['attack'],test_cluster_features['normal'],test_cluster_features['label'] 
#test_cluster_features = Imputer.fit_transform(Imputer(strategy="most_frequent",axis=0),test_cluster_features)

/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The functon distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:

In [5]:
train_cluster_features = train_cluster_features.astype(np.float32)
train_labels = train_labels.astype(np.float32)

test_cluster_features = test_cluster_features.astype(np.float32)
test_labels = test_labels.astype(np.float32)

import sklearn.preprocessing
train_cluster_features= sklearn.preprocessing.normalize(train_cluster_features,axis=0)
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(feature_array_train, train_label)
cluster_label_predict = gnb.predict(feature_array_test)


In [6]:
#cluster_label_predict
label_predict = [train_labels[x[0]] for x in zip(cluster_label_predict)]
label_actual = [test_labels[x[0]] for x in  zip(test_label)]

from sklearn.metrics import confusion_matrix
confusion_matrix(label_actual, label_predict) #0 = normal

array([[2758,   12],
       [  65,  132]])

In [9]:
X_test['pred'] = label_predict
X_test['act_label'] = label_actual
X_test.source.unique()

array(['131.202.240.218', '192.168.1.105', '192.168.1.101',
       '192.168.4.119', '192.168.3.115', '192.168.2.111', '192.168.4.118',
       '192.168.2.113', '0.0.0.0', '192.168.3.117', '192.168.3.116',
       '192.168.3.114', '192.168.2.112', '192.168.2.108', '192.168.2.107',
       '192.168.5.123', '192.168.2.110', '192.168.5.122', '192.168.2.106',
       '192.168.4.120', '192.168.1.102', '192.168.1.104', '192.168.2.109',
       '192.168.1.103', '192.168.4.121', '131.202.243.90',
       '64.237.127.131', '142.167.88.44', '63.111.123.26',
       '210.188.199.237'], dtype=object)

In [11]:
malicious_IP = X_test.groupby(['pred']).get_group(1)['source']
malicious_IP.unique()

array(['131.202.240.218', '192.168.1.105', '192.168.3.115',
       '192.168.4.118', '192.168.4.119', '0.0.0.0', '192.168.3.117',
       '192.168.3.114', '192.168.2.112', '192.168.2.108', '192.168.3.116',
       '192.168.5.123', '192.168.2.110', '192.168.2.107', '192.168.5.122',
       '192.168.2.106', '192.168.1.102', '192.168.1.104', '192.168.2.111',
       '192.168.1.101', '192.168.2.113', '192.168.1.103', '192.168.4.121',
       '192.168.2.109', '131.202.243.90', '192.168.4.120',
       '64.237.127.131', '63.111.123.26', '142.167.88.44',
       '210.188.199.237'], dtype=object)

In [12]:
malicious_act = X_test.groupby(['act_label']).get_group(1)['source']
malicious_act.unique()

array(['131.202.240.218', '192.168.1.105', '192.168.3.115', '0.0.0.0',
       '192.168.3.117', '192.168.3.116', '192.168.3.114', '192.168.2.112',
       '192.168.2.108', '192.168.2.107', '192.168.5.123', '192.168.2.110',
       '192.168.5.122', '192.168.2.106', '192.168.4.120', '192.168.1.102',
       '192.168.4.119', '192.168.1.104', '192.168.4.118', '192.168.1.101',
       '192.168.2.113', '192.168.1.103', '192.168.2.111', '192.168.4.121',
       '192.168.2.109', '131.202.243.90', '64.237.127.131',
       '142.167.88.44', '63.111.123.26', '210.188.199.237'], dtype=object)